In [2]:
import seaborn as sea
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing

sea.set(style='ticks')

In [4]:
card_data = pd.read_csv('creditcard_2023.csv')

In [5]:
card_data = card_data.drop(columns = 'id')
preprocessed = pd.DataFrame(preprocessing.scale(card_data.drop('Class', axis = 'columns')))
preprocessed['Class'] = card_data['Class']
preprocessed.columns = card_data.columns
x = preprocessed.columns.drop('Class')
y = ['Class']

In [6]:
logreg = LogisticRegression()
rfe = RFE(logreg)
rfe = rfe.fit(preprocessed[x], preprocessed[y].values.ravel())
print(rfe.support_)
print(rfe.ranking_)

[ True False  True  True False False  True  True  True  True  True  True
 False  True False  True  True  True False False False  True False False
 False False False False False]
[ 1  8  1  1 15  3  1  1  1  1  1  1 14  1  5  1  1  1 13 11  4  1  2  6
  7 12 10  9 16]


In [7]:
logreg = LogisticRegression()
rfe = RFE(logreg)
rfe = rfe.fit(preprocessed[x], preprocessed[y].values.ravel())
print(rfe.support_)
print(rfe.ranking_)


[ True False  True  True False False  True  True  True  True  True  True
 False  True False  True  True  True False False False  True False False
 False False False False False]
[ 1  8  1  1 15  3  1  1  1  1  1  1 14  1  5  1  1  1 13 11  4  1  2  6
  7 12 10  9 16]


In [8]:
list = rfe.support_.tolist()
final_variables = []
for index, i in enumerate(list):
    if i == True:
        final_variables.append(x[index])
final_variables

['V1',
 'V3',
 'V4',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V14',
 'V16',
 'V17',
 'V18',
 'V22']

In [9]:
X = preprocessed[final_variables]
y = card_data['Class']
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.144060
         Iterations 9
                          Results: Logit
Model:              Logit            Method:           MLE        
Dependent Variable: Class            Pseudo R-squared: 0.792      
Date:               2023-09-28 01:36 AIC:              163861.5868
No. Observations:   568630           BIC:              164019.1006
Df Model:           13               Log-Likelihood:   -81917.    
Df Residuals:       568616           LL-Null:          -3.9414e+05
Converged:          1.0000           LLR p-value:      0.0000     
No. Iterations:     9.0000           Scale:            1.0000     
--------------------------------------------------------------------
          Coef.    Std.Err.       z       P>|z|     [0.025    0.975]
--------------------------------------------------------------------
V1       -0.3546     0.0106    -33.4496   0.0000   -0.3754   -0.3338
V3       -0.3596     0.0116    -30.9643   0.0

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)
logreg = LogisticRegression(solver='lbfgs', max_iter=200)
logreg.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [11]:
y_train_predictions = logreg.predict(X_train)
y_test_predictions = logreg.predict(X_test)
y_train_predictions, y_test_predictions

(array([0, 1, 0, ..., 0, 1, 1], dtype=int64),
 array([1, 1, 1, ..., 0, 1, 0], dtype=int64))

In [12]:
lr_train_MSE = mean_squared_error(y_train, y_train_predictions)
lr_train_R2 = r2_score(y_train, y_train_predictions)

lr_test_MSE = mean_squared_error(y_test, y_test_predictions)
lr_test_R2 = r2_score(y_test, y_test_predictions)

results = pd.DataFrame(['Logistic Regression', lr_train_MSE, lr_test_MSE, lr_train_R2, lr_test_R2]).transpose()
results.columns = ['Model', 'Train MSE', 'Test MSE', 'Train R^2', 'Test R^2']
results

,Model,Train MSE,Test MSE,Train R^2,Test R^2
0,Logistic Regression,0.035094,0.035313,0.859625,0.858749
